<a href="https://www.kaggle.com/code/aleksandrmorozov123/python-mathematical-modeling?scriptVersionId=127620167" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

***This notebook about math modeling***

In [ ]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
from mpl_toolkits import mplot3d

# create the function for probability distributions
def plotNormalPDF_CDF_CHF ():
    mu = 10.0
    sigma = 1.0
    i = complex (0, 1)
    chf = lambda u: np.exp (i * mu * u - sigma * sigma *u * u / 2.0)
    pdf = lambda x: st.norm.pdf (x, mu, sigma)
    cdf = lambda x: st.norm.cdf (x, mu, sigma)
    
    x = np.linspace (10, 20, 200)
    u = np.linspace (0, 10, 250)
    # probability density function
    plt.figure (1)
    plt.plot (x, pdf (x))
    plt.grid ()
    plt.xlabel ('x')
    plt.ylabel ('PDF')
    # cumulative density fucntion
    plt.figure (3)
    ax = plt.axes (projection = '3d')
    chfV = chf (u)
    x = np.real (chfV)
    y = np.imag (chfV)
    ax.plot3D  (u, x, y, 'blue')
    ax.view_init (30, - 120)

plotNormalPDF_CDF_CHF ()


In [ ]:
# due bivariate_normal is deleted from package, I add the function
def bivariate_normal(X, Y, sigmax=1.0, sigmay=1.0,
                 mux=0.0, muy=0.0, sigmaxy=0.0):
    """
    Bivariate Gaussian distribution for equal shape *X*, *Y*.
    See `bivariate normal
    <http://mathworld.wolfram.com/BivariateNormalDistribution.html>`_
    at mathworld.
    """
    Xmu = X-mux
    Ymu = Y-muy

    rho = sigmaxy/(sigmax*sigmay)
    z = Xmu**2/sigmax**2 + Ymu**2/sigmay**2 - 2*rho*Xmu*Ymu/(sigmax*sigmay)
    denom = 2*np.pi*sigmax*sigmay*np.sqrt(1-rho**2)
    return np.exp(-z/(2*(1-rho**2))) / denom

# create the function for bivariate probaility function
def BivariateNormalPDFPlot ():
    # number of points in each direction
    n = 40;
    
    # denote the parameters
    mu_1 = 0;
    mu_2 = 0;
    sigma_1 = 1;
    sigma_2 = 0.5;
    rho1 = 0.0
    rho2 = - 0.8
    rho3 = 0.8
    
    # create a grid and a mulrivariate normal
    x = np.linspace (-3.0, 3.0, n)
    y = np.linspace (-3.0, 3.0, n)
    X, Y = np.meshgrid (x, y)
    Z = lambda rho: bivariate_normal(X, Y, sigma_1, sigma_2, mu_1, mu_2, rho*sigma_1*sigma_2)

    # make a 3D plot - rho = 0.0
    fig = plt.figure (1)
    ax = fig.add_subplot (projection = '3d')
    ax.plot_surface (X, Y, Z(rho1), cmap = 'viridis', linewidth = 0)
    ax.set_xlabel ('X axis')
    ax.set_ylabel ('Y axis')
    ax.set_zlabel ('Z axis')
    plt.show ()
    
    # make a 3D plot with rho = - 0.8
    fig = plt.figure (2)
    ax = fig.add_subplot (projection = '3d')
    ax.plot_surface (X, Y, Z (rho2), cmap = 'viridis', linewidth = 0)
    ax.set_xlabel ('X axis')
    ax.set_ylabel ('Y axis')
    ax.set_zlabel ('Z axis')
    plt.show ()
    
    # make a 3D plot with rho = 0.8
    fig = plt.figure (3)
    ax = fig.add_subplot (projection = '3d')
    ax.plot_surface (X, Y, Z (rho3), cmap = 'viridis', linewidth = 0)
    ax.set_xlabel ('X axis')
    ax.set_ylabel ('Y axis')
    ax.set_zlabel ('Z axis')
    plt.show ()

BivariateNormalPDFPlot ()
    

In [ ]:
# import required libraries
from scipy.interpolate import interp1d
from scipy import integrate

# create a random step function, approximating a stochastic fucntion g(t) with m = 10
def GeneratePathsGBMABM (NoOfPaths, NoOfSteps, T, r, sigma, S_0):
    Z = np.random.normal (0.0,1.0, [NoOfPaths, NoOfSteps])
    X = np.zeros ([NoOfPaths, NoOfSteps+1])
    time = np.zeros ([NoOfSteps + 1])
    
    X[:,0] = np.log (S_0)
    
    dt = T / float (NoOfSteps)
    for i in range (0, NoOfSteps):
        # making sure that samples from a normal have mean 0 and variance 1
        if NoOfPaths > 1: Z [:, 1] = (Z[:, i] - np.mean (Z[:, i])) / np.std (Z[:, i])
        X [:, i+1] = X [:, i] + (r - 0.5 * sigma * sigma) * dt + sigma * np.power (dt, 0.5) * Z [:, i]
        time [i+1] = time [i] + dt
    # compute exponent of ABM
    S = np.exp (X)
    paths = {'time':time, 'S':S}
    return paths

def mainCalculation ():
    NoOfPaths = 1
    NoOfSteps = 500
    T = 1
    r = 0.05
    sigma = 0.4
    S_0 = 100
    
    Paths = GeneratePathsGBMABM (NoOfPaths, NoOfSteps, T, r, sigma, S_0)
    timeGrid = Paths ['time']
    S = Paths ['S']
    
    plt.figure (1)
    plt.plot (timeGrid, np.transpose (S))
    plt.grid ()
    plt.xlabel ('time')
    plt.ylabel ('S(t)')
    
    m = 20
    pathNumber = 0
    f = interp1d (timeGrid, S [pathNumber, :])
    
    for k in range (1, m*T+1):
        tGrid = np.linspace (float ((k-1))/m, float (k)/m, 100)
        g_m =  m * integrate.trapezoid (np.squeeze (f(tGrid)), np.squeeze (tGrid))
        plt.plot (tGrid, g_m * np.ones ([len (tGrid), 1]), '-r', linewidth = 3.0)
        plt.plot (tGrid [0], g_m, 'or')
        plt.plot (tGrid [-1], g_m, 'or', markerfacecolor = [1, 1, 1])
    
mainCalculation ()

In [ ]:
# create function for integrated Brownian motion
def ComputeIntegrals (NoOfPaths, NoOfSteps, T):
    Z = np.random.normal (0.0, 1.0, [NoOfPaths, NoOfSteps])
    W = np.zeros ([NoOfPaths, NoOfSteps+1])
    I1 = np.zeros ([NoOfPaths, NoOfSteps+1])
    I2 = np.zeros ([NoOfPaths, NoOfSteps+1])
    time = np.zeros ([NoOfSteps+1])
    
    dt = T / float (NoOfSteps)
    for i in range (0, NoOfSteps):
        # making sure that samples from normal have mean 0 and variance 1
        if NoOfPaths > 1: Z[:, i] = (Z[:,i] - np.mean (Z[:, i])) / np.std (Z[:, i])
        W [:, i+1] = W [:, i] + np.power (dt, 0.5)*Z[:, i]
        I1[:, i+1] = I1[:, i] + W [:,i]*dt
        I2[:, i+1] = I2[:, i] + W[:, i]*(W[:,i+1]-W[:, i])
        time [i+1] = time[i] + dt
    paths = {'time':time, 'W':W, "I1":I1, 'I2':I2}
    return paths

NoOfPaths = 1
NoOfSteps = 1000
T = 1

W_t = ComputeIntegrals (NoOfPaths, NoOfSteps, 1)
timeGrid = W_t['time']
Ws = W_t['W']
intWsds = W_t['I1']
intWsdWs = W_t ['I2']

plt.figure (1)
plt.plot (timeGrid, np.transpose (Ws))
plt.plot (timeGrid, np.transpose (intWsds), 'r')
plt.plot (timeGrid, np.transpose (intWsdWs), 'k')
plt.grid ()
plt.xlabel ('time')
plt.ylabel ('value')
plt.title ('Integrated Brownian Motion paths')

In [ ]:
# import required libraries 
import scipy.stats as st

# create the fucntion
def PlotNormalDensity ():
    muV = [-1.0, -0.5, 0.0, 0.5, 1.0]
    sigmaV = [0.25, 0.75, 1.25, 1.75]
    
    # effect of mu for given sigma
    
    plt.figure (1)
    plt.grid ()
    plt.xlabel ('x')
    x = np.linspace (-5.0, 5.0, 250)
    
    for mu in muV:
        plt.plot (x, st.norm.pdf (x, mu, 1.0))
    plt.legend (['mu = -1.0', 'mu = -0.5', 'mu = 0.0', 'mu = 0.5', 'mu = 1.0'])
    
    # effect of sigma for given mu
    plt.figure (2)
    plt.grid ()
    plt.xlabel ('PDF')
    for sigma in sigmaV:
        plt.plot (x, st.norm.pdf (x, 0, sigma))
    plt.legend (['sigma = 0.25', 'sigma = 0.75', 'sigma = 1.25', 'sigma = 1.75'])
PlotNormalDensity ()

In [ ]:
# create Log normal density
def PlotLogNormalDensity ():
    muV = [-1.0, -0.5, 0.0, 0.5, 1.0]
    sigmaV = [0.25, 0.75, 1.25, 1.75]
    
    # effect of mu for a given sigma
    plt.figure (1)
    plt.grid ()
    plt.xlabel ('x')
    plt.xlabel ('PDF')
    x = np.linspace (0, 5.0, 250)
    
    for mu in muV:
        plt.plot (x, st.lognorm.pdf (x, scale = np.exp (mu), s = 1.0))
    plt.legend (['mu = -1.0', 'mu = -0.5', 'mu = 0.0', 'mu = 0.5', 'mu = 1.0'])
    
    # effect of sigma for a given mu
    plt.figure (2)
    plt.grid ()
    plt.xlabel ('PDF')
    for sigma in sigmaV:
        plt.plot (x, st.lognorm.pdf (x, scale = np.exp (0.0), s = sigma))
    plt.legend (['sigma = 0.25', 'sigma = 0.75', 'sigma = 1.25', 'sigma = 1.75',])

PlotLogNormalDensity ()

In [ ]:
def GeneratePathsGBM (NoOfPaths, NoOpfSteps, T, r, sigma, S_0):
    Z = np.random.normal (0.0, 1.0, [NoOfPaths, NoOfSteps])
    X = np.zeros ([NoOfPaths, NoOfSteps + 1])
    S = np.zeros ([NoOfPaths, NoOfSteps +1])
    time = np.zeros ([NoOfSteps + 1])
    
    X[:, 0] = np.log (S_0)
    
    dt = T / float (NoOfSteps)
    for i in range (0, NoOfSteps):
        # making sure that samples from a normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:, i] = (Z[:, i] - np.mean(Z[:, i])) / np.std (Z[:, i])
        X [:, i+1] = X [:, i] + (r - 0.5 * sigma * sigma) * dt + sigma *\
        np.power (dt, 0.5) * Z [:, i]
        time [i + 1] = time [i] + dt
    # compute exponent of ABM
    S = np.exp (X)
    paths = {'time': time, "S":S}
    return paths

def MainCode ():
    NoOfPaths = 8
    NoOfSteps = 1000
    S_0 = 1
    r = 0.05
    mu = 0.15
    sigma = 0.1 
    T = 10
    
    # money-savings account
    M = lambda t: np.exp (r * t)
    
    # Monte Carlo paths
    pathsQ  = GeneratePathsGBM (NoOfPaths, NoOfSteps, T, r, sigma, S_0)
    S_Q = pathsQ ["S"]
    pathsP = GeneratePathsGBM (NoOfPaths, NoOfSteps, T, mu, sigma, S_0)
    S_P = pathsP ['S']
    time = pathsQ ['time']
    
    # discounted stock paths
    S_Qdisc = np.zeros ([NoOfPaths, NoOfSteps + 1])
    S_Pdisc = np.zeros ([NoOfPaths, NoOfSteps + 1])
    i = 0
    
    for i, ti in enumerate (time):
        S_Qdisc [:, i] = S_Q [:, i]/M (ti)
        S_Pdisc [:, i] = S_P [:, i]/M (ti)
    
    # S(T)/M(T) with stock growing with rate r
    
    plt.figure (1)
    plt.grid ()
    plt.xlabel ('time')
    plt.ylabel ('S(T)')
    eSM_Q = lambda t: S_0 * np.exp (r * t) / M(t)
    plt.plot (time, eSM_Q (time), 'r--')
    plt.plot (time, np.transpose (S_Qdisc), 'blue')
    plt.legend (['E^Q [S(t)/M(t)]', 'paths S(t)/M(t)'])
    
    # S(T)/M(T) with stock growing with rate mu
    plt.figure (2)
    plt.grid ()
    plt.xlabel ('time')
    plt.ylabel ('S(t)')
    eSM_P = lambda t: S_0 * np.exp (mu * t) / M(t)
    plt.plot (time, eSM_P (time), 'r--')
    plt.plot (time, np.transpose (S_Pdisc), 'blue')
    plt.legend (['E^P[S(t)/M(t)]', 'paths S(t)/M(t)'])
    
MainCode ()

In [ ]:
#pathwise estimation for delta and vega for the Black-Scholes model 
# import required libraries 
import enum

# this class defines puts and calls
class OptionType (enum.Enum):
    CALL = 1.0
    PUT = -1.0
    
# Black-Scholes call option price
def BS_Call_Put_Option_Price (CP, S_0, K, sigma, t, T, r):
    K = np.array (K).reshape([len (K), 1])
    d1 = (np.log(S_0 / K) + (r + 0.5 * np.power (sigma, 2.0)) * (T - t)) / (sigma * np.sqrt (T - t))
    d2 = d1 - sigma * np.sqrt (t- T)
    if CP == OptionType.CALL:
        value = st.norm.cdf (d1) * S_0 - st.norm.cdf (d2) * K *np.excp (-r * (T - t))
    elif CP == OptionType.CALL:
        value = st.norm.cdg (-d2) * K * np.exp (-r * (T-t)) - st.norm.cdf (-d1)*S_0
    return value

def BS_Delta (CP, S_0, K, sigma, t, T, r):
    K = np.array (K).reshape ([len(K), 1])
    d1 = (np.log (S_0 / K) + (r + 0.5 * np.power (sigma,  2.0)) * (T-t)) / (sigma * np.sqrt (T-t))
    if CP == OptionType.CALL: 
        value = st.norm.cdf (d1)
    elif CP == OptionType.PUT:
        value = st.norm.cdf (d1) - 1
    return value

def BS_Gamma (S_0, K, sigma, t, T, r):
    K = np.array (K).reshape ([len(K), 1])
    d1 = (np.log (S_0 / K) + (r + 0.5 * np.power (sigma, 2.0)) * (T - t)) / (sigma * np.sqrt (T - t))
    return st.norm.pdf (d1) / (S_0 * digma * np.sqrt (T - t))
    
def BS_Vega (S_0, K, sigma, t, T, r):
    d1 = (np.log (S_0 / K) + (r + 0.5 * np.power (sigma, 2.0)) * (T - t)) / (sigma * np.sqrt (T -t))
    return S_0 * st.norm.pdf (d1) * np.sqrt (T - t)

def GeneratePathsGBMEuler (NoOfPaths, NoOfSteps, T, r, sigma, S_0):
    Z = np.random.normal (0.0, 1.0, [NoOfPaths, NoOfSteps])
    W = np.zeros ([NoOfPaths, NoOfSteps+1])
    
    # Approximation
    S = np.zeros ([NoOfPaths, NoOfSteps + 1])
    S [:, 0] = S_0
    X = np.zeros ([NoOfPaths, NoOfSteps+1])
    X[:, 0] = np.log (S_0)
    time = np.zeros ([NoOfSteps + 1])
    dt = T / float (NoOfSteps)
    for i in range (0, NoOfSteps):
        # mahking sure that smaples from a normal have mean 0 and variance 1
        if NoOfPaths > 1:
            Z[:, i] = (Z [:, i] - np.mean(Z[:, i])) / np.std (Z[:, i])
            W [:, i + 1] = W [:, i] + np.power (dt, 0.5) * Z [:, i]
            X [:, i + 1] = X [:, i] + (r - 0.5 * sigma ** 2.0) * dt + sigma * (W [:, i +1] - W [: ,i])
            time [i+1] = time [i] + dt
    # retern S
    paths = {"time": time, "S": np.exp (X)}
    return paths

def EUOptionPriceFromMCPathsGeneralized (Cp, S, K, T, r):
    # S is a vectior of Monte Carlo samples at T
    result = np.zeros ([len (K), 1])
    if CP == OptionType.CALL:
        for (idx, k) in enumerate (K):
            result[idx] = np.exp (-r*T) * np.mean (np.maximum (S-k, 0.0))
    elif CP == OptionTyo.PUT:
        for (idx, k) in enumerate (K):
            result [idx] = np.exp (-r*T)*np.mean (np.maximum (k-S, 0.0))
    return result

def PathwiseDelta (S0, S, K, r, T):
    temp1 = S[:, -1]>K
    return np.exp (-r*T)* np.mean (S[:, -1]/S0*temp1)

def PathwiseVega (S0, S, sigma, K, r, T):
    temp1 = S[:, -1]>K
    temp2 = 1.0/sigma * S [:, -1] * (np.log (S [:, -1]/ S0) - (r + 0.5 * sigma ** 2.0) *T)
    return np.exp (-r*T)* np.mean (temp1 * temp2)

def mainCalculation ():
    CP = OptionType.CALL
    S0 = 1
    r = 0.06
    sigma = 0.3
    T = 1
    K = np.array ([S0])
    t = 0.0
    NoOfSteps = 1000
    delta_Exact = BS_Delta (CP, S0, K, sigma, t, T, r)
    vega_Exact = BS_Vega (S0, K, sigma, t, T, r)
    NoOfPathsV = np.round (np.linspace (5, 20000, 50))
    deltaPathWiseV = np.zeros (len (NoOfPathsV))
    vegaPathWiseV = np.zeros (len (NoOfPathsV))
    
    for (idx, nPaths) in enumerate (NoOfPathsV):
        print ("Running simulation with {0} paths".format (nPaths))
        np.random.seed (3)
        paths1 = GeneratePathsGBMEuler (int (nPaths), NoOfSteps, T, r, sigma, S0)
        S = paths1["S"]
        delta_pathwise = PathwiseDelta (S0, S, K, r, T)
        deltaPathWiseV [idx] = delta_pathwise
        vega_pathwise = PathwiseVega (S0, S, sigma, K, r, T)
        vegaPathWiseV [idx] = vega_pathwise
        
    plt.figure (1)
    plt.grid ()
    plt.plot (NoOfPathsV, deltaPathWiseV, '.-r')
    plt.plot (NoOfPathsV, delta_Exact * np.ones([len (NoOfPathsV), 1]))
    plt.xlabel ('number of paths')
    plt.ylabel ("Delta")
    plt.title ('Convergence of pathwise delta w.r.t number of paths')
    plt.legend (['pathwise est', 'exact'])
    
    plt.figure (2)
    plt.grid ()
    plt.plot (NoOfPathsV, vegaPathWiseV, '.-r')
    plt.plot (NoOfPathsV, vega_Exact * np.ones ([len (NoOfPathsV), 1]))
    plt.xlabel ('number of paths')
    plt.ylabel ("Vega")
    plt.title ("Convergence of pathwise vega w.r.t number of paths")
    plt.legend (["pathwise est", "exact"])
    
mainCalculation ()

In [ ]:
# likelihood ratio estimation for delta and vega for the Black-Scholes model
# Additionaly the method is compared to pathwise differentiation

# import required libraries
import enum

# this class defines puts and calls
class OptionType (enum.Enum):
    CALL = 1.0
    PUT = -1.0
    
# Black-Scholes call option price
def BS_Call_Put_Price (CP, S_0, K, sigma, t, T, r):
    K = np.array (K).reshape ([len(K), 1])
    d1 = (np.log (S_0 / K) + (r + 0.5 * np.power (sigma, 2.0)) * (T -t)) / (sigma * np.sqrt (T -t))
    d2 = d1 - sigma * np.sqrt (T - t)
    if CP == OptionType.CALL: 
        value = st.norm.cdf(d1) * S_0 - st.norm.cdf (d2) * K * np.exp (-r * (T - t))
    elif  CP == OptionType.PUT:
        value = st.norm.cdf (-d) * K * np.exp (-r * (T - t)) - st.norm.cdf (-d1) * S_0
    return value

def BS_Delta (CP, S_0, K, sigma, t, T, r):
    K = np.array (K).reshape ([len (K), 1])
    d1 = (np.log (S_0 / K) + (r + 0.5 * np.power (sigma, 2.0)) * (T - t)) / (sigma * np.sqrt (T - t))
    if CP = OptionType.CALL:
        value = st.norm.cdf (d1)
    elif CP == OptionType.PUT:
        value = st.norm.cdf (d1)-1
    return value
    
def BS_Gamma (S_0, K, sigma, t, T, r):
    K = np.array (K).reshape ([len(K), 1])
    d1 = (np.log (S_0 / K) + (r + 0.5 * np.power (sigma, 2.0)) * (T - t)) / (sigma * np.sqrt (T - t))
    return st.norm.pdf (d1) / (S_0 * sigma * np.sqrt (T - t))

def BS_Vega (S_0, K, sigma, t, T, r):
    d1 = (np.log (S_0 / K) + ( r + 0.5 * np.power (sigma, 2.0)) * (T - t)) / (sigma * np.sqrt (T - t))
    return S_0  * st.norm.pdf (d1) * np.sqrt (T - t)

def GeneratePathsGBMEuler (NoOfPaths, NoOfSteos, T, r, sigma, S_0):
    Z = np.random.normal (0.0, 1.0, [NoOfPaths, NoOfSteps])
    W = np.zeros ([NoOfPaths, NoOfSteps + 1])